In [15]:
# Import all the necessary libraries
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2 as cv
import os
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input
from keras.models import Sequential, load_model
from keras.callbacks import TensorBoard

In [ ]:
# Load the classes
# class_names is a list of the name of the classes whose images are in the data folder
# class_names is automatically generated from the names of the subfolders in the data folder
# each subfolder in the data folder contains images of one class
# for example if the data folder contains subfolders named 'cats' and 'dogs', then class_names = ['cats', 'dogs']
# we save the number of classes in the variable num_classes for later use
class_names = [class_name for class_name in os.listdir('data') if os.path.isdir(os.path.join('data', class_name))]
num_classes = len(class_names)

# Load the data from the data folder and class_names as labels for the classes
# remap data as a dataset of (image, label) tuples, the image is divided by 255 to normalize the pixel values to the range [0, 1]
data = tf.keras.utils.image_dataset_from_directory('data', class_names=class_names)
data = data.map(lambda x, y: (x / 255, y))

# Split data into train, validation, and test sets
# the train set contains 60% of the data
# the validation set contains 30% of the data
# the test set contains the remaining 10% of the data
# the train, validation, and test sets are non-overlapping
# for train we take the first 60% of the data
# for val we skip the first 60% and take the next 30% of the data
# for test we skip the first 60% + 30% and take the remaining 10% of the data
train_size = int(len(data) * 0.6)
val_size = int(len(data) * 0.3)
test_size = len(data) - train_size - val_size
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)

# Define the input layer of the model, which is a 256x256x3 image
inputs = Input(shape=(256, 256, 3))

# Define the model, which is a sequential model, with the following layers:
# 1. A 3x3 convolutional layer with 16 filters and a ReLU activation function
# 2. A 2x2 max pooling layer
# 3. A 3x3 convolutional layer with 32 filters and a ReLU activation function
# 4. A 2x2 max pooling layer
# 5. A 3x3 convolutional layer with 16 filters and a ReLU activation function
# 6. A 2x2 max pooling layer
# 7. A flatten layer
# 8. A dense layer with 256 units and a ReLU activation function
# 9. A dense layer with num_classes units and a softmax activation function
model = Sequential([
    inputs,
    Conv2D(16, (3, 3), 1, activation='relu'),
    MaxPooling2D(),
    Conv2D(32, (3, 3), 1, activation='relu'),
    MaxPooling2D(),
    Conv2D(16, (3, 3), 1, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model, using the Adam optimizer, the sparse categorical crossentropy loss function, and the accuracy metric
# the Adam optimizer is a popular optimizer for training deep learning models
# the sparse categorical crossentropy loss function is a popular loss function for multi-class classification problems
# the accuracy metric is a popular metric for evaluating classification models
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Set up TensorBoard callback
logdir = 'logs'
tensorboard_callback = TensorBoard(log_dir=logdir)

# Fit the model, using the train set for training and the validation set for validation
# the model is trained for 20 epochs, and the TensorBoard callback is used to log training metrics
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test)

In [ ]:
# Plot training history, including loss and accuracy
plt.plot(hist.history['loss'], label='Training Loss')
plt.plot(hist.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(hist.history['accuracy'], label='Training Accuracy')
plt.plot(hist.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Save the model in the format: model_name.keras
model.save('model_name.keras')

In [ ]:
# To use the model in another script, you need to load it using the load_model function from keras.models
# Load the model
loaded_model = load_model('model_name.keras')

# Load the image we want to predict
# convert the image to RGB because OpenCV loads images in BGR format
# resize the image to 256x256, because the model expects images of this size
# normalize the image to the range [0, 1], because the model expects normalized images
# expand the dimensions of the image to (1, 256, 256, 3), because the model expects a batch of images
# a bunch of images is represented as a 4D tensor with shape (batch_size, height, width, channels)
# a single image is represented as a 4D tensor with shape (1, height, width, channels)
# this is because the model is designed to take a batch of images as input, even if the batch size is 1
# predict the class of the image using the model
# then do what you want with the prediction, the prediction will be a list of probabilities for each class
image = cv.imread('test1.jpg')
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
resized = cv.resize(image, (256, 256))
normalized = resized / 255.0
expanded = np.expand_dims(normalized, axis=0)
predict = model.predict(expanded)